In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import librosa
import librosa.display
import IPython.display as pld
import warnings
warnings.filterwarnings('ignore')
import csv
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf

In [ ]:
def energy(signal,frame_length,hop_length):
    energy=[]
    for i in range(0,len(signal),hop_length):
        rms_current_frame_energy=np.sum(signal[i:i+frame_length]**2)
        energy.append(rms_current_frame_energy)
    total_energy=np.array(energy)
    total_energy_mean=np.mean(total_energy)
    return total_energy_mean

def rms(signal,frame_length,hop_length):
    for i in range(0,len(signal),hop_length):
        rms_energy_current_frame=np.sqrt(np.sum(signal[i:i+frame_length]**2)/frame_length)
    return rms_energy_current_frame


In [ ]:
emotions={
  'W':'anger',
  'L':'boredom',
  'E':'disgust',
  'A':'fear',
  'F':'happyness',
  'T':'sadness',
  'N':'neutral',
  }

from scipy import signal
sr=16000
q = signal.windows.hamming(320)

In [ ]:
# load the data set
for dirname, _, filenames in os.walk('/content/drive/MyDrive/DATASETS/EmoDB Dataset/wav'):

    for filename in filenames:
        paths = os.path.join(dirname, filename)
        label = filename.split('_')[0]  # it will split based on _ and extract the last word or first from right side
        label=emotions[label[-2]]
        # i.e OAF_back_angry.wav ---> wll give now angry.wav
        print(label)
        # it will split the label based on dot(.) and take the first word i.e fear.wav-->fear
        # print(label.lower()) # it will conver all the word into lowercase
        # print(paths)
        signal, sample_rate = librosa.load(paths)
        #signal = librosa.effects.preemphasis(signal, coef=0.97, zi=None, return_zf=False)
        # normalize the audio signal
        signal = librosa.util.normalize(signal)
        # total energy of the audio signal
        # total_energy = energy(signal, frame_length=320, hop_length=160)
        # # print(total_energy)
        # hop_length = 160
        # frame_length = 320
        # threshold_energy = 0.06 * total_energy
        # #print(threshold_energy)
        # new_signal = []
        # for i in range(0, len(signal), hop_length):
        #     current_frame = signal[i:i + frame_length]
        #     # print(len(current_frame))
        #     current_frame_energy = np.sum(signal[i:i + frame_length] ** 2)
        #     # print(current_frame_energy)
        #     current_frame1 = np.zeros(320)
        #     if current_frame_energy >= threshold_energy:
        #         for index in range(len(current_frame)):
        #             current_frame1[index] = current_frame[index] * q[index]
        #         new_signal.extend(current_frame1)
        #         # for ix in current_frame1:
        #         # new_signal.append(ix)
        #
        # new_signal = np.array(signal)
        mel_spectro = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=320, hop_length=160, n_mels=40).T
        mel_spectro = np.mean(librosa.power_to_db(mel_spectro), axis=0)
        MFCCs = librosa.feature.mfcc(y=signal, n_fft=320, hop_length=160, sr=sr, n_mfcc=40).T
        MFCCs_mean = np.mean(MFCCs, axis=0)  # 20 in size

        # print(mel_spectro.shape)
        spectro = librosa.stft(y=signal, n_fft=320, hop_length=160).T  # stft of the speech signal
        spectro_abs = np.abs((spectro) ** 2)  # linear spectrogram of speech signal
        spectro_db = np.mean(librosa.power_to_db(spectro_abs), axis=0)

        delta_MFCCs = np.mean((librosa.feature.delta(MFCCs, mode='nearest')), axis=0)
        delta2_MFCCs = np.mean(librosa.feature.delta(MFCCs, order=2, mode='nearest'), axis=0)
        stft = np.abs(librosa.stft(signal))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, n_fft=320, hop_length=160, sr=sr).T, axis=0)  # 12 in size
        # mel = np.mean(librosa.feature.melspectrogram(new_signal, n_fft=512, hop_length=512, sr=sr).T, axis=0)  # 128 in size
        tonnetz = np.mean(librosa.feature.tonnetz(y=signal, sr=sr).T, axis=0)
        spec = np.mean(librosa.feature.spectral_contrast(y=signal, sr=22050, n_fft=320, hop_length=160).T, axis=0)

        # to calculate the zero crossing rate features of each frame using formula directly

        rms_energy = np.mean(librosa.feature.rms(y=signal))

        data = []
        data.append(filename)
        for i in MFCCs_mean:
            data.append(i)
        for i1 in delta_MFCCs:
            data.append(i1)
        for i2 in delta2_MFCCs:
            data.append(i2)
        for l in mel_spectro:
            data.append(l)
        for m in spectro_db:
            data.append(m)
        for j in chroma:
            data.append(j)
        # for k in mel:
        #     data.append(k)
        for n in tonnetz:
            data.append(n)
        for o in spec:
            data.append(o)
        data.append(rms_energy)
        data.append(label)

        with open('/content/drive/MyDrive/DATASETS/EmoDB Dataset/feature_extracted.csv',"a",newline="") as f:
            writer = csv.writer(f)
            writer.writerow(data)

        #print(filename)

print('data set is loaded')

sadness
fear
boredom
happyness
anger
happyness
anger
boredom
anger
sadness
anger
anger
anger
boredom
disgust
disgust
anger
fear
disgust
anger
fear
fear
happyness
happyness
happyness
neutral
sadness
happyness
anger
boredom
disgust
boredom
boredom
neutral
boredom
sadness
disgust
sadness
sadness
boredom
disgust
boredom
happyness
disgust
anger
fear
neutral
anger
anger
boredom
sadness
happyness
happyness
disgust
sadness
disgust
boredom
fear
fear
anger
sadness
anger
boredom
anger
disgust
sadness
boredom
neutral
fear
boredom
anger
anger
anger
boredom
fear
happyness
sadness
anger
fear
fear
fear
boredom
anger
anger
neutral
disgust
disgust
happyness
anger
neutral
sadness
boredom
boredom
boredom
happyness
happyness
neutral
neutral
happyness
neutral
fear
neutral
boredom
happyness
anger
disgust
fear
disgust
sadness
fear
disgust
sadness
neutral
anger
sadness
neutral
happyness
neutral
fear
happyness
sadness
anger
anger
neutral
fear
anger
anger
fear
sadness
boredom
happyness
neutral
fear
boredom
sadne